# The Power of Compound Interest

This notebook will explore how compound interest grows investments over time. Use the cells below to run simulations, visualize growth, and compare different contribution strategies.
- In this experiment, we will see two investors: one that invests $10,000 at once and leaves the money to grow, and one that starts with $1,000, but continually invests $100 a month.
- Each investor will be have their money invested only into the S&P 500 for 50 years, resembling scenarios for young investors
- Their results will be graphed and interpreted at the end, and we will see who ends up making more money.


## The Big Question

What happens when steady contributions in the S&P 500 meet compound interest over decades?

## Research

The S&P 500 is a stock market index that tracks the 500 largest publicly traded US companies. It's often used as a benchmark for the overall performance of the US stock market. In short, it's how the economy is doing in terms of major businesses. Historically, since 1928, the S&P 500 has returned about 10-11% per year on average before inflation and about 7% per year on average after inflation. In 2024, last year's annual return was +24.2%, including dividends. In this experiment, we use the S&P 500 because it's the easiest way to show compound interest.

## Hypothesis

If an investor invested a large sum at one time and let compound interest do the rest, then that investor would make more over 50 years than an investor who starts with a smaller amount but consistently invests a small sum each month.

## Formulas

For the one-time investor, we will use the basic compound interest formula:

$$

FV = P(1+r)^t

$$

And for the consistent investor, we'll use a bit more advanced of a formula:

$$

FV=PMT×r(1+r)n−1​

$$
Here's what everything stands for:
- FV - Future value - the total amount after compounding
- P - Principle - the initial amount you start with
- r - rate - Interest rate per period
- n - time - total amount of compounding periods
- PMT - Regular payment added each period(only for second formula)

In [6]:
# Interactive two-investor simulator (one-time vs monthly contributions)
# Integrates interactive sliders into the earlier simulation cell.
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Try to import ipywidgets; if not available, we'll show a static example and instructions
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    WIDGETS_AVAILABLE = True
except Exception:
    WIDGETS_AVAILABLE = False


def fv_one_time(P, r, t, n=1):
    """Future value of a one-time principal P after t years with annual rate r (as decimal) and n compounding per year."""
    return P * (1 + r / n) ** (n * t)


def fv_with_contributions(P, PMT, r, t, n=1):
    """Future value with end-of-period contributions (PMT) each compounding period.

    r is annual rate as decimal, t in years, n periods per year.
    """
    if np.isclose(r, 0.0):
        return P + PMT * (n * t)
    periods = int(n * t)
    r_period = r / n
    # vector for each period (end-of-period contributions)
    times = np.arange(0, periods + 1)
    fv = P * (1 + r_period) ** times
    # contributions: PMT applied at end of each period (exclude time 0)
    contrib = np.zeros_like(fv, dtype=float)
    contrib[1:] = PMT * ((1 + r_period) ** (times[1:]) - 1) / r_period if not np.isclose(r_period, 0.0) else PMT * times[1:]
    return fv + contrib


def plot_two_investors(P_A, PMT_A, P_B, PMT_B, annual_rate_pct, years, compounding):
    annual_rate = annual_rate_pct / 100.0
    n = compounding
    periods = np.arange(0, int(years * n) + 1)
    times_years = periods / n

    # compute series for plotting
    values_A = P_A * (1 + annual_rate / n) ** periods
    # For contributions, compute value at each period
    if np.isclose(annual_rate, 0.0):
        values_B = P_B + PMT_B * periods
    else:
        r_period = annual_rate / n
        # contributions at end of each period
        contrib = np.zeros_like(periods, dtype=float)
        contrib[1:] = PMT_B * (((1 + r_period) ** (periods[1:]) - 1) / r_period)
        values_B = P_B * (1 + r_period) ** periods + contrib

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(times_years, values_A, label=f'Investor A (one-time ${P_A:,})', lw=2)
    ax.plot(times_years, values_B, label=f'Investor B (start ${P_B:,} + ${PMT_B}/period)', lw=2)
    ax.set_xlabel('Years')
    ax.set_ylabel('Portfolio value ($)')
    ax.set_title(f'Growth over {years} years at {annual_rate_pct:.2f}% annual return (compounded {n}x/year)')
    ax.legend()
    ax.grid(True)
    import matplotlib.ticker as mtick
    ax.yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
    plt.show()

    # Yearly snapshot
    idx = np.arange(0, len(periods), n)
    yearly = pd.DataFrame({
        'year': (periods[idx] / n).astype(int),
        'one_time': values_A[idx],
        'monthly_contrib': values_B[idx]
    })
    yearly['one_time'] = yearly['one_time'].map('${:,.2f}'.format)
    yearly['monthly_contrib'] = yearly['monthly_contrib'].map('${:,.2f}'.format)
    display(yearly.head(16))


if WIDGETS_AVAILABLE:
    # Sliders and controls for both investors
    P_A_widget = widgets.IntSlider(value=10_000, min=0, max=200_000, step=100, description='P_A', continuous_update=True)
    PMT_A_widget = widgets.IntSlider(value=0, min=0, max=5_000, step=10, description='PMT_A', continuous_update=True)
    P_B_widget = widgets.IntSlider(value=1_000, min=0, max=200_000, step=100, description='P_B', continuous_update=True)
    PMT_B_widget = widgets.IntSlider(value=100, min=0, max=5_000, step=10, description='PMT_B', continuous_update=True)
    rate_widget = widgets.FloatSlider(value=7.0, min=0.0, max=20.0, step=0.1, description='Rate (%)', continuous_update=True)
    years_widget = widgets.IntSlider(value=40, min=1, max=100, step=1, description='Years', continuous_update=True)
    compounding_widget = widgets.Dropdown(options=[('Annually',1), ('Semiannually',2), ('Quarterly',4), ('Monthly',12), ('Daily',365)], value=12, description='Compounding')

    out = widgets.Output()

    def on_change(change=None):
        with out:
            clear_output(wait=True)
            P_A = P_A_widget.value
            PMT_A = PMT_A_widget.value
            P_B = P_B_widget.value
            PMT_B = PMT_B_widget.value
            rate = rate_widget.value
            years = years_widget.value
            n = compounding_widget.value
            print(f'Parameters: A: P={P_A}, PMT={PMT_A}; B: P={P_B}, PMT={PMT_B}; rate={rate}% years={years} n={n}')
            plot_two_investors(P_A, PMT_A, P_B, PMT_B, rate, years, n)

    # Observe changes (value changes will trigger immediate updates because continuous_update=True)
    for w in [P_A_widget, PMT_A_widget, P_B_widget, PMT_B_widget, rate_widget, years_widget, compounding_widget]:
        w.observe(on_change, names='value')

    controls = widgets.VBox([
        widgets.HBox([P_A_widget, PMT_A_widget]),
        widgets.HBox([P_B_widget, PMT_B_widget]),
        widgets.HBox([rate_widget, years_widget, compounding_widget])
    ])

    display(controls)
    display(out)
    # Initialize
    on_change()
else:
    print('ipywidgets is not available in your environment.')
    print('To enable interactive sliders, run:')
    print('pip install ipywidgets')
    print('\nShowing a static example:')
    example_params = dict(P_A=10000, PMT_A=0, P_B=1000, PMT_B=100, annual_rate_pct=7.0, years=40, compounding=12)
    plot_two_investors(**example_params)


Output()

## Analysis & Conclusion

After running the simulation, we  see how each investment strategy performed over the 40-year period.

- Investor A totaled out with approvimately $160,000
- Investor B totaled out with almost $300,000

You'd think that just relying on compound interest would make you a millionaire by the title. But we see that consistently adding more contributions outperforms plain compound interest. This experiment shows that, while compound interest is a powerful factor with investing, it isn't everything. It will only get you so far. Regular contributions are necessary to really build wealth over time. Combine that with other investments like gold or real estate and smart investing, I can almost guarantee you will be a millionaire one day. Thank you for reading!
